In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

from tqdm import tqdm #optional, if you do not want to import remove tqdm() from loops!

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Preprocessing

We again will be using the MNIST dataset. This time I prepared the dataset as a npy file. We will load the data visualize an example and the implement logistic regression.

In [3]:
# change if the file is in a different directory
f_features = "features.npy"

# change if the file is in a different directory
f_labels = "labels.npy"

# load the data
features=np.load(f_features)
labels=np.load(f_labels)

In [4]:
plt.imshow(features[0,:,:], cmap='gray')

In [5]:
batch_index = 4800
offset = 600
validation_index = batch_index + offset
test_index = validation_index + offset
# As you know we need to split the data into training, validation and test
x_train=features[0:batch_index,:,:]
x_train=x_train.reshape((batch_index, 784))
y_train=labels[0:batch_index].astype(int)

x_val=features[batch_index:validation_index,:,:]
x_val=x_val.reshape((offset, 784))
y_val=labels[batch_index:validation_index].astype(int)

x_test=features[validation_index:test_index,:,:]
x_test=x_test.reshape((offset, 784))
y_test=labels[validation_index:test_index].astype(int)

# Note: Normally the split has to be random and stratified for the validation set and random for the test set

In [6]:
plt.imshow(x_train[0].reshape(28,28), cmap='gray')

# 2. Logistic Regression

From the lecture we know that logistic regression is given by affined transformation of the data followed by applying the sigmoid function. Our first step is to implement the function we need.

In [7]:
def layer(x, w):
    """


    Parameters
    ----------
    x : feature tensor of dimension (N,M)
    w : learnable parameters of dimension (M+1, C)

    N is the number of samples, M the number of features and C the number of classes.

    Returns
    -------
    res : output tensor of dimension (N, C)

    res should be the result of the matrix multiplication of an expanded feature tensor (1 column) with
    the learnable parameters.

    """


    x_bias = np.column_stack([x, np.ones((x.shape[0], 1))])
    res = x_bias @ w

    return x_bias, res

In [8]:
# test your code
w=np.ones((28*28+1,10)) # 28*28 are the number of features and the bias leads to +1
res=layer(x_train, w)
res

(array([[0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.]]),
 array([[ 78.36470687,  78.36470687,  78.36470687, ...,  78.36470687,
          78.36470687,  78.36470687],
        [114.62745199, 114.62745199, 114.62745199, ..., 114.62745199,
         114.62745199, 114.62745199],
        [140.68627563, 140.68627563, 140.68627563, ..., 140.68627563,
         140.68627563, 140.68627563],
        ...,
        [167.23921684, 167.23921684, 167.23921684, ..., 167.23921684,
         167.23921684, 167.23921684],
        [105.94117744, 105.94117744, 105.94117744, ..., 105.94117744,
         105.94117744, 105.94117744],
        [ 78.42353026,  78.42353026,  78.42353026, ...,  78.42353026,
          78.42353026,  78.42353026]]))

## 2.1 Loss Function
In exercise sheet 0, we just guessed values, but now we are smarter! First we need to define an appropriate loss function. The dataset has ten target classes, so we want to implement cross-entropy loss:

$\mathcal{L}=\sum_{y}1\{\hat{y}=y\}(-\log[p(y)])$ 

The $p(y)$ is given by the softmax function

$p(y_i)=\frac{e^{x_i}}{\sum_ie^{x_i}}$

So the softmax should return a vector representing the probability of each class.

In [9]:
def compute_loss(x,w,y):
    """
    Parameters
    -----------
    x : feature tensor of dimension (N,M+1)
    w : learnable parameters of dimension (M+1, C)
    y : target tensor

    Returns
    -------
    loss function specified above
    """
    scores = x @ w
    probs = softmax(scores)
    log_likelihood = -np.log(probs[range(len(y)), y])
    return np.sum(log_likelihood) / len(y)

In [10]:
def softmax(y):
    """
    

    Parameters
    ----------
    y : Prediction tensor of dimension (N, C). 

    Returns
    -------
    res : Softmax transformed tensor of dimension (N,C)

    res should be the result of the softmax transformation of y.

    """
    exp_y = np.exp(y-np.max(y, axis=1, keepdims=True))
    return exp_y/exp_y.sum(axis=1, keepdims=True)

In [11]:
def model(x, w):
    """
    
    Parameters
    ----------
    x : feature tensor of dimension (N,M)
    w : learnable parameters of dimension (M+1, C)

    N is the number of samples, M the number of features and C the number of classes.

    Returns
    -------
    res : Prediction tensor of dimension (N,1)

    res should be the classification of our model 

    """

    x_bias, logits = layer(x, w)

    prob = softmax(logits)

    res = np.argmax(prob, axis=1).reshape(-1, 1)

    return res

## 2.2 Optimization
We have already learned about optimization algorithms. In this exercise we want to learn more about gradient descent, stochastic gradient descent and Newton’s method.

### 2.2.1 Gradient Descent
For gradient descent we need to updates our parameters using the steepest descent of the gradient with respect to the parameter. It is given by the equation:

$w_{n+1}=w_n-\epsilon_n\nabla\mathcal{L(w_n)}$

$\epsilon_n$ is the learning rate and a hyperparameter of our optimization approach. We can calculate the gradient by using the composition rule for derivatives.

The challenge is to broadcast the right dimensions!

In [12]:
def classTensor(y, C):
    """
    
    Parameters
    ----------
    y : class vector of dimension N containing the true classes
    C : number of classes


    Returns
    -------
    res : class tensor of dimension (N,C)

    We want to transform the vector into a binary tensor. If res_ij=1 then it means that at sample i we have class j. Otherwise 
    res_ij=0.

    """  

    N = len(y)
    res = np.zeros((N, C))

    for i in range(N):
        res[i, y[i]] = 1

    return res

In [13]:
def gradientDescent(x, y, w, lr):
    """
    
    Parameters
    ----------
    x : feature tensor of dimension (N,M)
    y : class vector of dimension N containing the true classes
    w : learnable parameters of dimension (M+1, C)
    lr: learning rate of the gradient descent

    N is the number of samples, M the number of features and C the number of classes.

    Returns
    -------
    w : updated learnable parameters

    """
    x_bias, logits = layer(x, w)

    prob_logits = softmax(logits)

    y_one_hot = classTensor(y, w.shape[1])

    gradient = x_bias.T @ (prob_logits - y_one_hot)
    w  = w - lr * gradient

    return w

### 2.2.2 Stochastic Gradient Descent
In this section implement stochastic gradient descent by writing the function "def stochasticGradientDescent(...)"

In [14]:
def stochasticGradientDescent(x, y, w, lr, batch_size):
   """

    Parameters
    ----------
    x : feature tensor of dimension (N,M)
    y : class vector of dimension N containing the true classes
    w : learnable parameters of dimension (M+1, C)
    lr: learning rate of the stochastic gradient descent
    batch_size : batch size of the stochastic gradient descent

    N is the number of samples, M the number of features and C the number of classes.

    Returns
    -------
    w : updated learnable parameters

    """
   N = x.shape[0]
   indices = np.random.permutation(N)

   for i in range(0, N-batch_size, batch_size):
       batch_indices = indices[i:i + batch_size]
       x_batch = x[batch_indices]
       y_batch = y[batch_indices]

       x_bias, logits = layer(x_batch, w)

       prob_logits = softmax(logits)

       y_one_hot = classTensor(y_batch, w.shape[1])

       gradient = x_bias.T @ (prob_logits - y_one_hot)

       w = w - lr * gradient

   return w

### 2.2.3 Newton’s method
In this section implement Newton's method by writing the function "def newtonMethod(...)"

In [15]:
def newtonMethod(x, y, w, max_iter=100, tol=1e-3, reg_lambda=1e-5, show=False):
    """
    Parameters
    ----------
    x : feature tensor of dimension (N,M)
    y : class vector of dimension N containing the true classes
    w : learnable parameters of dimension (M+1, C)
    max_iter : maximum number of iterations
    tol : tolerance for convergence

    N is the number of samples, M the number of features and C the number of classes.

    Returns
    -------
    w : updated learnable parameters

    """
    validation_acc = []
    train_acc = []

    N, M = x.shape
    M += 1 # bias term
    C = w.shape[1]
    losses = []
    for iteration  in tqdm(range(max_iter)):
        x_bias, logits = layer(x, w)

        prob_logits = softmax(logits)

        y_one_hot = classTensor(y, C)

        g = x_bias.T @ (prob_logits - y_one_hot)

        loss = compute_loss(x_bias, w, y)
        losses.append(loss)

        H = x_bias.T @ x_bias + reg_lambda * np.eye(M) # we do this for numerical stability
        H_inv = np.linalg.inv(H)

        w_update = H_inv @ g
        w = w - w_update


        # This here is not part of the training method and it's only for the visualisation and validation
        # I know it's implemented poorly and it could cause issues easily, but it was the easiest way to implement the visualisation
        if show:
            train_acc.append(accuracy_score(y_train, model(x_train, w)))
            validation_acc.append(accuracy_score(y_val, model(x_val,w)))
        if loss <= tol or np.linalg.norm(g) < tol:
            print(f"Converged at iteration {iteration}")
            break

    if show:
        return w, losses, train_acc, validation_acc
    return w, losses

In [16]:
# Testing
limit = np.sqrt(6 / (785 + 10))
w = np.random.uniform(-0.01, 0.01, (785, 10))
w, losses = newtonMethod(x_train, y_train, w)

100%|██████████| 100/100 [00:05<00:00, 18.59it/s]


In [17]:
fig, ax = plt.subplots(figsize=(6, 5))

ax.set_xlabel("Epochs")
ax.set_ylabel("Accuracy")

ax.plot(losses, c="orange", label="val", lw=1.5)
ax.grid(color='gray', linestyle='dashed', alpha=0.3)
ax.legend(loc="lower right", fontsize=11)

# 3. Training
Now use the MNIST dataset to train a classifier and compare the results.

# 3.1 Train and Plot Gradient Descent

In [18]:
acc_train=[]
acc_val=[]

n_epochs=100
learningRate=0.001

w = np.random.uniform(-0.01, 0.01, (785, 10))

for e in tqdm(range(n_epochs)):

    w = gradientDescent(x_train, y_train, w, learningRate)
    pred = model(x_train, w)

    acc_train.append(accuracy_score(y_train, pred))
    acc_val.append(accuracy_score(y_val, model(x_val,w)))

100%|██████████| 100/100 [00:01<00:00, 54.57it/s]


In [19]:
fig, ax = plt.subplots(figsize=(6, 5))


ax.set_xlabel("Epochs")
ax.set_ylabel("Accuracy")

ax.plot(acc_val, c="orange", label="val", lw=1.5)
ax.plot(acc_train, c="blue", label="train",lw=1.5)
ax.grid(color='gray', linestyle='dashed', alpha=0.3)
ax.legend(loc="lower right", fontsize=11)

## 3.2 Implement training for stochastic gradient descent

In [20]:
sgd_train_acc = []
sgd_val_acc = []

lr = 0.001
n_epochs = 100
batch_size = 32

w = np.random.uniform(-0.01, 0.01, (785, 10))


for e in tqdm(range(n_epochs)):

    w = stochasticGradientDescent(x_train, y_train, w, lr, batch_size)
    pred = model(x_train, w)

    sgd_train_acc.append(accuracy_score(y_train, pred))
    sgd_val_acc.append(accuracy_score(y_val, model(x_val,w)))

100%|██████████| 100/100 [00:02<00:00, 45.00it/s]


In [21]:
fig, ax = plt.subplots(figsize=(6, 5))


ax.set_xlabel("Epochs")
ax.set_ylabel("Accuracy")

ax.plot(sgd_val_acc, c="orange", label="val", lw=1.5)
ax.plot(sgd_train_acc, c="blue", label="train",lw=1.5)
ax.grid(color='gray', linestyle='dashed', alpha=0.3)
ax.legend(loc="lower right", fontsize=11)

## 3.3 Implement training for newton's method

In [22]:
w = np.random.uniform(-0.01, 0.01, (785, 10))
w, losses, newton_train_acc, newton_val_acc = newtonMethod(x_train, y_train, w, show=True)

100%|██████████| 100/100 [00:05<00:00, 18.07it/s]


In [23]:
fig, ax = plt.subplots(figsize=(6, 5))


ax.set_xlabel("Epochs")
ax.set_ylabel("Accuracy")

ax.plot(newton_val_acc, c="orange", label="val", lw=1.5)
ax.plot(newton_train_acc, c="blue", label="train",lw=1.5)
ax.grid(color='gray', linestyle='dashed', alpha=0.3)
ax.legend(loc="lower right", fontsize=11)